# Air Quality Prediction for Embedded Systems

This notebook demonstrates the process of building, quantizing, and evaluating lightweight machine learning models for air quality prediction, suitable for deployment on embedded systems. We use a popular public dataset, design two compact models, convert them to TensorFlow Lite, and compare their performance and size.

## 1. Dataset Preparation

We use the [UCI Air Quality Dataset](https://archive.ics.uci.edu/ml/datasets/Air+Quality) which contains sensor data for air quality prediction. The dataset is publicly available and widely used for this task.

In [ ]:
# Download and load the UCI Air Quality dataset
import pandas as pd
import numpy as np
import requests
from io import BytesIO
import zipfile

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip'
r = requests.get(url)
z = zipfile.ZipFile(BytesIO(r.content))
df = pd.read_csv(z.open('AirQualityUCI.csv'), sep=';', decimal=',')

df = df.iloc[:, :-2]
df = df.dropna()

for col in df.columns[2:]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df = df.dropna()

df.head()

## 2. Data Preprocessing

We select relevant features and the target variable ("C6H6(GT)" - Benzene concentration). Data is split into train and test sets, and normalized for model training.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Select features and target
features = [
    'CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)', 'PT08.S2(NMHC)',
    'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)',
    'PT08.S5(O3)', 'T', 'RH', 'AH'
]
target = 'C6H6(GT)'

X = df[features].values
y = df[target].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print('Train shape:', X_train.shape, 'Test shape:', X_test.shape)

## 3. Model Design

We design two compact neural network models suitable for embedded deployment:
- **Model 1:** A small fully connected (Dense) neural network
- **Model 2:** A small 1D Convolutional Neural Network (CNN)

Both models are designed to have a small number of parameters (target: <60KB after quantization).

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Model 1: Small Dense NN
def build_dense_model(input_shape):
    model = keras.Sequential([
        layers.Input(shape=input_shape),
        layers.Dense(16, activation='relu'),
        layers.Dense(8, activation='relu'),
        layers.Dense(1)
    ])
    return model

# Model 2: Small 1D CNN
def build_cnn_model(input_shape):
    model = keras.Sequential([
        layers.Reshape((input_shape[0], 1), input_shape=input_shape),
        layers.Conv1D(8, 3, activation='relu'),
        layers.Flatten(),
        layers.Dense(8, activation='relu'),
        layers.Dense(1)
    ])
    return model

dense_model = build_dense_model((X_train.shape[1],))
cnn_model = build_cnn_model((X_train.shape[1],))

dense_model.summary()
cnn_model.summary()

## 4. Model Training

We train both models using mean squared error loss and early stopping to avoid overfitting.

In [ ]:
callbacks = [keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]

dense_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
cnn_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

history_dense = dense_model.fit(
    X_train, y_train, validation_split=0.1, epochs=50, batch_size=32, callbacks=callbacks, verbose=2
)
history_cnn = cnn_model.fit(
    X_train, y_train, validation_split=0.1, epochs=50, batch_size=32, callbacks=callbacks, verbose=2
)

## 5. Model Conversion to TensorFlow Lite

We convert both models to TensorFlow Lite format with post-training quantization to minimize size for embedded deployment.

In [ ]:
import os
os.makedirs('Models', exist_ok=True)

def convert_to_tflite(model, model_name, quantize=True):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    if quantize:
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        def representative_dataset():
            for i in range(100):
                yield [X_train[i:i+1].astype(np.float32)]
        converter.representative_dataset = representative_dataset
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        converter.inference_input_type = tf.int8
        converter.inference_output_type = tf.int8
    tflite_model = converter.convert()
    with open(f'Models/{model_name}.tflite', 'wb') as f:
        f.write(tflite_model)
    return tflite_model

dense_model.save('Models/dense_model.h5')
cnn_model.save('Models/cnn_model.h5')

tflite_dense = convert_to_tflite(dense_model, 'dense_model_quant')
tflite_cnn = convert_to_tflite(cnn_model, 'cnn_model_quant')

def get_file_size(path):
    return os.path.getsize(path) / 1024  # KB

sizes = {
    'Dense Original': get_file_size('Models/dense_model.h5'),
    'Dense TFLite': get_file_size('Models/dense_model_quant.tflite'),
    'CNN Original': get_file_size('Models/cnn_model.h5'),
    'CNN TFLite': get_file_size('Models/cnn_model_quant.tflite'),
}
sizes

## 6. Model Evaluation

We evaluate both the original and quantized models on the test set and compare their accuracy and size.

In [ ]:
dense_eval = dense_model.evaluate(X_test, y_test, verbose=0)
cnn_eval = cnn_model.evaluate(X_test, y_test, verbose=0)

print(f"Dense Model - Test MAE: {dense_eval[1]:.3f}")
print(f"CNN Model   - Test MAE: {cnn_eval[1]:.3f}")

In [ ]:
def evaluate_tflite_model(tflite_path, X_test, y_test):
    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    y_preds = []
    for i in range(len(X_test)):
        x = X_test[i:i+1].astype(np.float32)
        scale, zero_point = input_details[0]['quantization']
        x_q = (x / scale + zero_point).astype(np.int8)
        interpreter.set_tensor(input_details[0]['index'], x_q)
        interpreter.invoke()
        y_q = interpreter.get_tensor(output_details[0]['index'])
        y_pred = (y_q.astype(np.float32) - output_details[0]['quantization'][1]) * output_details[0]['quantization'][0]
        y_preds.append(y_pred[0][0])
    y_preds = np.array(y_preds)
    mae = np.mean(np.abs(y_preds - y_test))
    return mae

dense_tflite_mae = evaluate_tflite_model('Models/dense_model_quant.tflite', X_test, y_test)
cnn_tflite_mae = evaluate_tflite_model('Models/cnn_model_quant.tflite', X_test, y_test)

print(f"Dense TFLite Model - Test MAE: {dense_tflite_mae:.3f}")
print(f"CNN TFLite Model   - Test MAE: {cnn_tflite_mae:.3f}")

### Model Comparison Table

Below is a summary of model sizes and test MAE for all models.

In [ ]:
import pandas as pd

results = pd.DataFrame({
    'Model': ['Dense Original', 'Dense TFLite', 'CNN Original', 'CNN TFLite'],
    'Size (KB)': [sizes['Dense Original'], sizes['Dense TFLite'], sizes['CNN Original'], sizes['CNN TFLite']],
    'Test MAE': [dense_eval[1], dense_tflite_mae, cnn_eval[1], cnn_tflite_mae]
})
results

## 7. Visualization

We visualize the performance and size of the models using bar plots and error distributions.

In [ ]:
import matplotlib.pyplot as plt

# 1. Model Size Comparison
plt.figure(figsize=(6,4))
plt.bar(results['Model'], results['Size (KB)'], color=['#4e79a7','#f28e2b','#76b7b2','#e15759'])
plt.ylabel('Model Size (KB)')
plt.title('Model Size Comparison')
plt.xticks(rotation=15)
plt.tight_layout()
plt.savefig('Models/model_size_comparison.png')
plt.show()

In [ ]:
# 2. Test MAE Comparison
plt.figure(figsize=(6,4))
plt.bar(results['Model'], results['Test MAE'], color=['#4e79a7','#f28e2b','#76b7b2','#e15759'])
plt.ylabel('Test MAE')
plt.title('Test MAE Comparison')
plt.xticks(rotation=15)
plt.tight_layout()
plt.savefig('Models/model_mae_comparison.png')
plt.show()

In [ ]:
# 3. Error Distribution for Best Model (CNN TFLite)
cnn_tflite_preds = []
interpreter = tf.lite.Interpreter(model_path='Models/cnn_model_quant.tflite')
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
for i in range(len(X_test)):
    x = X_test[i:i+1].astype(np.float32)
    scale, zero_point = input_details[0]['quantization']
    x_q = (x / scale + zero_point).astype(np.int8)
    interpreter.set_tensor(input_details[0]['index'], x_q)
    interpreter.invoke()
    y_q = interpreter.get_tensor(output_details[0]['index'])
    y_pred = (y_q.astype(np.float32) - output_details[0]['quantization'][1]) * output_details[0]['quantization'][0]
    cnn_tflite_preds.append(y_pred[0][0])
cnn_tflite_preds = np.array(cnn_tflite_preds)

plt.figure(figsize=(6,4))
plt.hist(cnn_tflite_preds - y_test, bins=50, color='#76b7b2', alpha=0.8)
plt.xlabel('Prediction Error')
plt.ylabel('Frequency')
plt.title('Error Distribution (CNN TFLite)')
plt.tight_layout()
plt.savefig('Models/cnn_tflite_error_distribution.png')
plt.show()

## 8. Conclusion

- Both models achieve reasonable accuracy with very small model sizes after quantization (<60KB).
- The quantized models are suitable for deployment on embedded systems.
- The CNN model slightly outperforms the dense model in this task.
- All visualizations and models are saved in the `Models/` directory.